In [1]:
import pandas as pd

In [15]:
raw_us_deaths = pd.read_csv("data/RAW_us_deaths.csv")
maskData = pd.read_csv("data/mask-use-by-county.csv")
maskMandateData = pd.read_csv("data/U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv")
elPasoDeaths = raw_us_deaths[(raw_us_deaths["Province_State"] == "Colorado") & (raw_us_deaths["Admin2"] == "El Paso")]
print(elPasoDeaths.head())

    Province_State   Admin2       UID iso2 iso3  code3    FIPS Country_Region  \
277       Colorado  El Paso  84008041   US  USA    840  8041.0             US   

           Lat       Long_  ... 10/21/22  10/22/22  10/23/22  10/24/22  \
277  38.832401 -104.525141  ...     1805      1805      1805      1806   

     10/25/22  10/26/22  10/27/22  10/28/22  10/29/22  10/30/22  
277      1806      1806      1808      1808      1808      1808  

[1 rows x 1025 columns]


In [6]:
colData = elPasoDeaths[elPasoDeaths.columns[0:12]]
print(colData.head())

    Province_State   Admin2       UID iso2 iso3  code3    FIPS Country_Region  \
277       Colorado  El Paso  84008041   US  USA    840  8041.0             US   

           Lat       Long_           Combined_Key  Population  
277  38.832401 -104.525141  El Paso, Colorado, US      720403  


In [13]:
pasoDeathsByDate = elPasoDeaths[elPasoDeaths.columns[12:]]
pasoDeathsByDate = pd.DataFrame(pasoDeathsByDate.iloc[0])
pasoDeathsByDate = pasoDeathsByDate.reset_index()
pasoDeathsByDate.columns = ["date", "deaths"]
print(pasoDeathsByDate)

colFips = colData["FIPS"].values[0]
print(colFips)

          date  deaths
0      1/22/20       0
1      1/23/20       0
2      1/24/20       0
3      1/25/20       0
4      1/26/20       0
...        ...     ...
1008  10/26/22    1806
1009  10/27/22    1808
1010  10/28/22    1808
1011  10/29/22    1808
1012  10/30/22    1808

[1013 rows x 2 columns]
8041.0


In [19]:
elPasoMaskUse = maskData[maskData["COUNTYFP"] == colFips]
print(elPasoMaskUse.head())

elPasoMaskMandates = maskMandateData[(maskMandateData["State_Tribe_Territory"] == "CO") & (maskMandateData['County_Name'] == 'El Paso County')]
print(elPasoMaskMandates.head())
elPasoMaskMandates["Face_Masks_Required_in_Public"] = elPasoMaskMandates["Face_Masks_Required_in_Public"].fillna("No")
print(elPasoMaskMandates.head())

     COUNTYFP  NEVER  RARELY  SOMETIMES  FREQUENTLY  ALWAYS
265      8041  0.031   0.089      0.081       0.196   0.603
      State_Tribe_Territory     County_Name  FIPS_State  FIPS_County  \
70454                    CO  El Paso County           8           41   
70484                    CO  El Paso County           8           41   
70485                    CO  El Paso County           8           41   
70493                    CO  El Paso County           8           41   
70494                    CO  El Paso County           8           41   

            date  order_code Face_Masks_Required_in_Public Source_of_Action  \
70454  5/26/2020           2                           NaN              NaN   
70484  5/27/2020           2                           NaN              NaN   
70485  5/28/2020           2                           NaN              NaN   
70493  4/10/2020           2                           NaN              NaN   
70494  4/11/2020           2                        

C:\Users\ecorp\AppData\Local\Temp\ipykernel_24504\3460426010.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elPasoMaskMandates["Face_Masks_Required_in_Public"] = elPasoMaskMandates["Face_Masks_Required_in_Public"].fillna("No")
